# 部署到联机终结点

若要使用应用程序中的模型，可以将该模型部署到联机终结点。 你将从本地文件创建 MLflow 模型并测试终结点。

## 准备工作

需要最新版本的 azureml-ai-ml 包才能运行此笔记本中的代码。 运行下面的单元以验证是否已安装它。

> **注意**：
> 如果未安装 azure-ai-ml 包，请运行 `pip install azure-ai-ml` 以进行安装。

In [ ]:
## 连接到工作区

安装必需的 SDK 包后，就可以连接到工作区了。

若要连接到工作区，我们需要标识符参数 - 订阅 ID、资源组名称和工作区名称。 已为你填写资源组名称和工作区名称。 只需订阅 ID 即可完成命令。

若要查找所需的参数，请单击工作室右上角的订阅和工作区名称。 右侧将打开一个窗格。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 复制订阅 ID，并将“YOUR-SUBSCRIPTION-ID”替换为复制的值。 </p>

## 定义并创建终结点

最终目标是将模型部署到终结点。 因此，首先需要创建终结点。 终结点将是一个 HTTPS 终结点，应用程序可以调用该终结点来接收模型的预测。 应用程序可以通过使用终结点的 URI，并使用密钥或令牌进行身份验证来使用终结点。

运行以下单元以定义终结点。 请注意，终结点的名称必须是唯一的。 你将使用 `datetime` 函数生成唯一名称。

In [ ]:
接下来，通过运行以下单元来创建终结点。 此过程可能需要几分钟时间。 创建终结点时，可以了解[什么是 Azure 机器学习终结点](https://learn.microsoft.com/azure/machine-learning/concept-endpoints)。

In [ ]:
<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 重要说明！ 等待终结点创建成功后再继续！ 工作室中应显示绿色通知。 </p>

## 配置部署

可以将多个模型部署到一个终结点。 这主要适用于更新已部署的模型，同时将当前模型保留在生产环境中的情况。 你需要配置部署，以指定需要部署到终结点的模型。 在下面的单元中，你将引用经过训练并存储在本地 `model` 文件夹中的模型（存储在此笔记本所在的文件夹中）。 请注意，由于使用的是 MLflow 模型，因此无需指定环境或评分脚本。

你还将指定要部署的模型所需的基础结构。

## 创建部署

最后，可以通过运行以下单元将模型实际部署到终结点：

In [ ]:
部署模型可能需要 10-15 分钟。 在等待部署模型时，可详细了解[此视频中的托管终结点](https://www.youtube.com/watch?v=SxFGw_OBxNM&ab_channel=MicrosoftDeveloper)。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 重要说明！ 请等到部署完成后再继续！ 工作室中应显示绿色通知。</p>

由于只有一个模型部署到终结点，因此你希望此部署占用 100% 的流量。 如果将多个模型部署到终结点，则可以使用相同的方法在部署的模型之间分配流量。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 重要说明！ 等到蓝色部署配置后再继续！ 工作室中应显示绿色通知。 </p> 

## 测试部署

通过调用终结点来测试部署的模型。 包含示例数据的 JSON 文件用作输入。 经过训练的模型根据医疗数据（如年龄、BMI 和怀孕次数）预测患者是否患有糖尿病。 `[0]` 表示患者没有糖尿病。 `[1]` 表示患者患有糖尿病。

In [ ]:
（可选）可以更改 `sample-data.json` 文件中的值，尝试获取不同的预测。

## 列出终结点

尽管可以在工作室中查看所有终结点，但也可以使用 SDK 列出所有终结点：

In [ ]:
## 获取终结点详细信息

如果需要有关特定终结点的详细信息，也可以使用 SDK 浏览详细信息。

## 删除终结点和部署

由于终结点始终可用，因此无法通过暂停终结点来节省成本。 若要避免不必要的成本，请删除终结点。

In [ ]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

The deployment of the model may take 10-15 minutes. While waiting for the model to be deployed, you can learn more about [managed endpoints in this video](https://www.youtube.com/watch?v=SxFGw_OBxNM&ab_channel=MicrosoftDeveloper).

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the deployment is completed before continuing! A green notification should appear in the studio.</p>

Since you only have one model deployed to the endpoint, you want this deployment to take 100% of the traffic. If you deploy multiple models to the endpoint, you could use the same approach to distribute traffic across the deployed models.

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the blue deployment is configured before continuing! A green notification should appear in the studio. </p> 

## Test the deployment

Let's test the deployed model by invoking the endpoint. A JSON file with sample data is used as input. The trained model predicts whether a patient has diabetes or not, based on medical data like age, BMI, and the number of pregnancies. A `[0]` indicates a patient doesn't have diabetes. A `[1]` means a patient does have diabetes.

In [ ]:
# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="sample-data.json",
)

if response[1]=='1':
    print("Diabetic")
else:
    print ("Not diabetic")

Optionally, you can change the values in the `sample-data.json` file to try and get a different prediction.

## List endpoints

Although you can view all endpoints in the Studio, you can also list all endpoints using the SDK:

In [ ]:
endpoints = ml_client.online_endpoints.list()
for endp in endpoints:
    print(endp.name)

## Get endpoint details

If you want more information about a specific endpoint, you can explore the details using the SDK too.

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

## Delete the endpoint and deployment

As an endpoint is always available, it can't be paused to save costs. To avoid unnecessary costs, delete the endpoint.

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)